##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Observação: você pode executar este exemplo agora mesmo num notebook estilo Jupyter, sem necessidade de configuração! Basta clicar em "Executar no Google Colab"

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/model_analysis/tfma_basic.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/model_analysis/tfma_basic.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/model_analysis/tfma_basic.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

# TensorFlow Model Analysis

***Exemplo de um componente chave do TensorFlow Extended (TFX)***

O [TensorFlow Model Analysis (TFMA)](https://www.tensorflow.org/tfx/guide/tfma) é uma biblioteca para realizar avaliação de modelos em diferentes fatias de dados. O TFMA realiza suas computações de maneira distribuída em grandes quantidades de dados usando o [Apache Beam](https://beam.apache.org/documentation/programming-guide/) .

Este notebook colab de exemplo ilustra como o TFMA pode ser usado para investigar e visualizar o desempenho de um modelo em relação às características do dataset. Usaremos um modelo que treinamos anteriormente e agora você pode brincar com os resultados! O modelo que treinamos foi para o [exemplo Chicago Taxi](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline), que usa o [dataset Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) da cidade de Chicago. Explore o dataset completo na [IU do BigQuery](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips) .

Como modelador e desenvolvedor, pense em como esses dados são usados ​​e nos possíveis benefícios e danos que as previsões de um modelo podem causar. Um modelo como este poderia reforçar um viés social e disparidades. Uma determinada característica é relevante para o problema que você quer resolver ou será que ela vai introduzir um viés? Para mais informações, leia sobre <a target="_blank" href="https://developers.google.com/machine-learning/fairness-overview/">Equidade em ML</a>.

Observação: Para entender o TFMA e como ele funciona com o Apache Beam, você precisará saber um pouco sobre o próprio Apache Beam. O <a target="_blank" href="https://beam.apache.org/documentation/programming-guide/">Guia de programação do Beam</a> é um ótimo lugar para começar.

As colunas do dataset são:

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

## Instale extensões do Jupyter

Observação: se estiver executando num notebook Jupyter local, essas extensões do Jupyter deverão ser instaladas no ambiente antes de rodar o Jupyter.

```bash
jupyter nbextension enable --py widgetsnbextension --sys-prefix
jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix
```

## Instale o TensorFlow Model Analysis (TFMA)

Isto vai baixar todas as dependências e levará um minuto.


In [ ]:
# Upgrade pip to the latest, and install TFMA.
!pip install -U pip
!pip install tensorflow-model-analysis

**Agora você precisa reiniciar o runtime antes de executar as células abaixo.**

In [ ]:
# This setup was tested with TF 2.10 and TFMA 0.41 (using colab), but it should
# also work with the latest release.
import sys

# Confirm that we're using Python 3
assert sys.version_info.major==3, 'This notebook must be run using Python 3.'

import tensorflow as tf
print('TF version: {}'.format(tf.__version__))
import apache_beam as beam
print('Beam version: {}'.format(beam.__version__))
import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.__version__))

**OBSERVAÇÃO: A saída acima deve estar livre de erros antes de continuar. Execute novamente a instalação se ainda estiver vendo erros. Além disso, certifique-se de reiniciar o runtime/kernel antes de passar para a próxima etapa.**

## Carregue os arquivos

Faremos download de um arquivo tar que contém tudo o que precisamos. Ele inclui:

- Datasets de treinamento e avaliação
- Esquema de dados
- Modelos salvos para treinamento e serviço (Keras e Estimator) e modelos salvos de avaliação (Estimator).

In [ ]:
# Download the tar file from GCP and extract it
import io, os, tempfile
TAR_NAME = 'saved_models-2.2'
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, TAR_NAME, 'data')
MODELS_DIR = os.path.join(BASE_DIR, TAR_NAME, 'models')
SCHEMA = os.path.join(BASE_DIR, TAR_NAME, 'schema.pbtxt')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

!curl -O https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/{TAR_NAME}.tar
!tar xf {TAR_NAME}.tar
!mv {TAR_NAME} {BASE_DIR}
!rm {TAR_NAME}.tar

print("Here's what we downloaded:")
!ls -R {BASE_DIR}

## Processe o esquema

Entre as coisas que baixamos havia um esquema para nossos dados que foi criado pelo [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started/). Vamos processá-lo agora para que possamos usá-lo com o TFMA.

In [ ]:
import tensorflow as tf
from google.protobuf import text_format
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow.core.example import example_pb2

schema = schema_pb2.Schema()
contents = file_io.read_file_to_string(SCHEMA)
schema = text_format.Parse(contents, schema)

## Use o esquema para criar TFRecords

Precisamos dar ao TFMA acesso ao nosso dataset, então vamos criar um arquivo TFRecords. Podemos usar nosso esquema para criá-lo, pois ele nos fornece o tipo correto para cada característica.

In [ ]:
import csv

datafile = os.path.join(DATA_DIR, 'eval', 'data.csv')
reader = csv.DictReader(open(datafile, 'r'))
examples = []
for line in reader:
  example = example_pb2.Example()
  for feature in schema.feature:
    key = feature.name
    if feature.type == schema_pb2.FLOAT:
      example.features.feature[key].float_list.value[:] = (
          [float(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.INT:
      example.features.feature[key].int64_list.value[:] = (
          [int(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.BYTES:
      example.features.feature[key].bytes_list.value[:] = (
          [line[key].encode('utf8')] if len(line[key]) > 0 else [])
  # Add a new column 'big_tipper' that indicates if tips was > 20% of the fare. 
  # TODO(b/157064428): Remove after label transformation is supported for Keras.
  big_tipper = float(line['tips']) > float(line['fare']) * 0.2
  example.features.feature['big_tipper'].float_list.value[:] = [big_tipper]
  examples.append(example)

tfrecord_file = os.path.join(BASE_DIR, 'train_data.rio')
with tf.io.TFRecordWriter(tfrecord_file) as writer:
  for example in examples:
    writer.write(example.SerializeToString())

!ls {tfrecord_file}

## Configure e execute o TFMA

O TFMA suporta vários tipos de modelos diferentes, incluindo modelos TF keras, modelos baseados em APIs genéricas do TF2, bem como modelos baseados em estimadores TF. O guia [get_started](https://www.tensorflow.org/tfx/model_analysis/get_started) contém a lista completa de tipos de modelos suportados e quaisquer restrições. Neste exemplo, mostraremos como configurar um modelo baseado em Keras, bem como um modelo baseado em Estimator que foi salvo como [`EvalSavedModel`](https://www.tensorflow.org/tfx/model_analysis/eval_saved_model). Veja o [FAQ](https://www.tensorflow.org/tfx/model_analysis/faq) para exemplos de outras configurações.

O TFMA fornece suporte para calcular métricas que foram usadas no momento do treinamento (ou seja, métricas integradas), bem como métricas definidas após o modelo ser salvo como parte das definições de configuração do TFMA. Para a [configuração](https://www.tensorflow.org/tfx/model_analysis/setup) do Keras, demonstraremos a adição manual de nossas métricas e gráficos como parte de nossa configuração (consulte o guia de [métricas](https://www.tensorflow.org/tfx/model_analysis/metrics) para obter informações sobre as métricas e gráficos suportados). Para a configuração do Estimator usaremos as métricas integradas que foram salvas com o modelo. Nossas configurações também incluem uma série de especificações de fatiamento que são discutidas com mais detalhes nas seções a seguir.

Depois de criar um [`tfma.EvalConfig`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig) e [`tfma.EvalSharedModel`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalSharedModel) podemos então executar o TFMA usando [`tfma.run_model_analysis`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/run_model_analysis). Isso vai criar um [`tfma.EvalResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult) que podemos usar posteriormente para renderizar nossas métricas e gráficos.

### Keras

In [ ]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
keras_eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # For keras (and serving models) we need to add a `label_key`.
    label_key: "big_tipper"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "AUC" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_values: {
      key: "trip_start_month"
      value: "1"
    }
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our keras model.
keras_model_path = os.path.join(MODELS_DIR, 'keras', '2')
keras_eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=keras_model_path,
    eval_config=keras_eval_config)

keras_output_path = os.path.join(OUTPUT_DIR, 'keras')

# Run TFMA
keras_eval_result = tfma.run_model_analysis(
    eval_shared_model=keras_eval_shared_model,
    eval_config=keras_eval_config,
    data_location=tfrecord_file,
    output_path=keras_output_path)

### Estimator

In [ ]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
estimator_eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # To use EvalSavedModel set `signature_name` to "eval".
    signature_name: "eval"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_values: {
      key: "trip_start_month"
      value: "1"
    }
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our eval saved model.
estimator_base_model_path = os.path.join(
    MODELS_DIR, 'estimator', 'eval_model_dir')
estimator_model_path = os.path.join(
    estimator_base_model_path, os.listdir(estimator_base_model_path)[0])
estimator_eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=estimator_model_path,
    eval_config=estimator_eval_config)

estimator_output_path = os.path.join(OUTPUT_DIR, 'estimator')

# Run TFMA
estimator_eval_result = tfma.run_model_analysis(
    eval_shared_model=estimator_eval_shared_model,
    eval_config=estimator_eval_config,
    data_location=tfrecord_file,
    output_path=estimator_output_path)

## Visualizando métricas e gráficos

Agora que executamos a avaliação, vamos dar uma olhada em nossas visualizações usando o TFMA. Para os exemplos a seguir, visualizaremos os resultados da execução da avaliação no modelo Keras. Para visualizar o modelo baseado no Estimator, atualize o `eval_result_path` para apontar para nossa variável `estimator_output_path`.

In [ ]:
eval_result_path = keras_output_path
# eval_result_path = estimator_output_path

eval_result = keras_eval_result
# eval_result = estimator_eval_result

### Renderizando métricas

O TFMA fornece APIs de dataframe em [`tfma.experimental.dataframe`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/experimental) para carregar a saída materializada como [`Pandas DataFrames`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). Para visualizar as métricas você pode usar `metrics_as_dataframes(tfma.load_metrics(eval_path))`, que retorna um objeto que potencialmente contém diversos DataFrames, um para cada tipo de valor de métrica (`double_value`, `confusion_matrix_at_thresholds`, `bytes_value` e `array_value`). Os DataFrames específicos preenchidos dependem do resultado da avaliação. Aqui, mostramos o DataFrame `double_value` como exemplo.

In [ ]:
import tensorflow_model_analysis.experimental.dataframe as tfma_dataframe
dfs = tfma_dataframe.metrics_as_dataframes(
  tfma.load_metrics(eval_result_path))

display(dfs.double_value.head())

Cada um dos DataFrames possui uma coluna multi-índice com as colunas de nível superior: `slices`, `metric_keys` e `metric_values`​​. As colunas exatas de cada grupo podem mudar de acordo com a carga útil. podemos usar a API `DataFrame.columns` para inspecionar todas as colunas multi-índice. Por exemplo, as colunas de fatias são 'Overall', 'trip_start_day', 'trip_start_hour' e 'trip_start_month', que são configuradas por `slicing_specs` no `eval_config`.

In [ ]:
print(dfs.double_value.columns)

### Auto pivot

O DataFrame é detalhado por design para que não haja perda de informações da carga útil. No entanto, às vezes, para consumo direto, podemos querer organizar as informações de uma forma mais concisa, mas com perdas: fatias como linhas e métricas como colunas. O TFMA fornece uma API `auto_pivot` para essa finalidade. O utilitário passa por todas as colunas não exclusivas dentro de `metric_keys` e condensa todas as fatias numa única coluna `stringified_slices` por padrão.

In [ ]:
tfma_dataframe.auto_pivot(dfs.double_value).head()

### Filtrando fatias

Como as saídas são DataFrames, qualquer API DataFrame nativa pode ser usada para dividir o DataFrame. Por exemplo, se estivermos interessados ​​apenas em `trip_start_hour` de 1, 3, 5, 7 e não em `trip_start_day`, podemos usar a lógica de filtragem `.loc` do DataFrame. Novamente, usamos a função `auto_pivot` para reorganizar o DataFrame na visualização fatia vs. métricas depois da filtragem.

In [ ]:
df_double = dfs.double_value
df_filtered = (df_double
  .loc[df_double.slices.trip_start_hour.isin([1,3,5,7])]
)
display(tfma_dataframe.auto_pivot(df_filtered))

### Ordenando por valores de métrica

Também podemos ordenar as fatias por valor de métrica. Como exemplo, mostramos como ordenar fatias no DataFrame acima, de acordo com o AUC ascendente, para que possamos encontrar fatias com desempenho insatisfatório. Isto envolve duas etapas: auto-pivot para que as fatias sejam representadas como linhas e colunas como métricas e, em seguida, a ordenação do DataFrame pivotizado, pela coluna AUC.

In [ ]:
# Pivoted table sorted by AUC in ascending order.
df_sorted = (
    tfma_dataframe.auto_pivot(df_double)
    .sort_values(by='auc', ascending=True)
    )
display(df_sorted.head())

### Renderizando gráficos

Quaisquer gráficos que foram adicionados ao `tfma.EvalConfig` como `metric_specs` pós-treinamento podem ser exibidos usando [`tfma.view.render_plot`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_plot).

Tal como acontece com as métricas, os gráficos podem ser visualizados por fatia. Ao contrário das métricas, apenas gráficos para um valor de fatia específico podem ser exibidos, portanto, `tfma.SlicingSpec` deve ser usado e deve especificar o nome e o valor da característica da fatia. Se nenhuma fatia for fornecida, os gráficos da fatia `Overall` serão usados.

No exemplo abaixo, estamos exibindo os gráficos `CalibrationPlot` e `ConfusionMatrixPlot` que foram calculados para a fatia `trip_start_hour:1`.

In [ ]:
tfma.view.render_plot(
    eval_result,
    tfma.SlicingSpec(feature_values={'trip_start_hour': '1'}))

## Acompanhando o desempenho do modelo ao longo do tempo

Seu dataset de treinamento será usado para treinar seu modelo e, espera-se, será representativo de seu dataset de teste e dos dados que serão enviados ao seu modelo em produção. No entanto, embora os dados nas solicitações de inferência possam permanecer iguais aos seus dados de treinamento, em muitos casos eles poderão começar a mudar o suficiente para que o desempenho do seu modelo seja alterado.

Isso significa que você precisa monitorar e medir o desempenho do seu modelo continuamente, para poder estar ciente e reagir às mudanças. Vamos dar uma olhada em como a TFMA pode ajudar.

Vamos carregar três execuções de modelos diferentes e usar o TFMA para ver como eles se comparam usando [`render_time_series`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_time_series).

In [ ]:
# Note this re-uses the EvalConfig from the keras setup.

# Run eval on each saved model
output_paths = []
for i in range(3):
  # Create a tfma.EvalSharedModel that points at our saved model.
  eval_shared_model = tfma.default_eval_shared_model(
      eval_saved_model_path=os.path.join(MODELS_DIR, 'keras', str(i)),
      eval_config=keras_eval_config)

  output_path = os.path.join(OUTPUT_DIR, 'time_series', str(i))
  output_paths.append(output_path)

  # Run TFMA
  tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                          eval_config=keras_eval_config,
                          data_location=tfrecord_file,
                          output_path=output_path)

Primeiro, imaginaremos que treinamos e implantamos nosso modelo ontem e agora queremos ver como ele está se saindo com os novos dados que chegam hoje. A visualização começará mostrando o AUC. Na IU, você pode:

- Adicionar outras métricas usando o menu "Add metric series".
- Fechar gráficos indesejados clicando no x
- Passe o mouse sobre os pontos de dados (os finais dos segmentos de linha no gráfico) para obter mais detalhes

Observação: Nos gráficos de séries de métricas, o eixo X é o nome do diretório do modelo da execução do modelo que você está examinando. Esses nomes em si não são significativos.

In [ ]:
eval_results_from_disk = tfma.load_eval_results(output_paths[:2])

tfma.view.render_time_series(eval_results_from_disk)

Agora vamos imaginar que mais um dia se passou e queremos ver como está o desempenho dos novos dados que chegam hoje, em comparação com os dois dias anteriores:

In [ ]:
eval_results_from_disk = tfma.load_eval_results(output_paths)

tfma.view.render_time_series(eval_results_from_disk)

## Validação de modelos

O TFMA pode ser configurado para avaliar vários modelos ao mesmo tempo. Normalmente, isto é feito para comparar um novo modelo contra uma referência (como o modelo atualmente em serviço) para determinar quais são as diferenças de desempenho nas métricas (por exemplo, AUC, etc.) em relação à linha de referência. Quando [limites](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold) (thresholds) são configurados, o TFMA produzirá um registro [`tfma.ValidationResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/ValidationResult) indicando se o desempenho corresponde às expectativas.

Vamos reconfigurar nossa avaliação Keras para comparar dois modelos: um candidato e uma referência. Também validaremos o desempenho do candidato em relação à referência, definindo um [`tmfa.MetricThreshold`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold) na métrica AUC.

In [ ]:
# Setup tfma.EvalConfig setting
eval_config_with_thresholds = text_format.Parse("""
  ## Model information
  model_specs {
    name: "candidate"
    # For keras we need to add a `label_key`.
    label_key: "big_tipper"
  }
  model_specs {
    name: "baseline"
    # For keras we need to add a `label_key`.
    label_key: "big_tipper"
    is_baseline: true
  }

  ## Post training metric information
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "BinaryAccuracy" }
    metrics { class_name: "BinaryCrossentropy" }
    metrics {
      class_name: "AUC"
      threshold {
        # Ensure that AUC is always > 0.9
        value_threshold {
          lower_bound { value: 0.9 }
        }
        # Ensure that AUC does not drop by more than a small epsilon
        # e.g. (candidate - baseline) > -1e-10 or candidate > baseline - 1e-10
        change_threshold {
          direction: HIGHER_IS_BETTER
          absolute { value: -1e-10 }
        }
      }
    }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_keys: ["trip_start_month"]
  }
  slicing_specs {
    feature_keys: ["trip_start_hour", "trip_start_day"]
  }
""", tfma.EvalConfig())

# Create tfma.EvalSharedModels that point at our keras models.
candidate_model_path = os.path.join(MODELS_DIR, 'keras', '2')
baseline_model_path = os.path.join(MODELS_DIR, 'keras', '1')
eval_shared_models = [
  tfma.default_eval_shared_model(
      model_name=tfma.CANDIDATE_KEY,
      eval_saved_model_path=candidate_model_path,
      eval_config=eval_config_with_thresholds),
  tfma.default_eval_shared_model(
      model_name=tfma.BASELINE_KEY,
      eval_saved_model_path=baseline_model_path,
      eval_config=eval_config_with_thresholds),
]

validation_output_path = os.path.join(OUTPUT_DIR, 'validation')

# Run TFMA
eval_result_with_validation = tfma.run_model_analysis(
    eval_shared_models,
    eval_config=eval_config_with_thresholds,
    data_location=tfrecord_file,
    output_path=validation_output_path)

Ao executar avaliações com um ou mais modelos em relação a uma referência, o TFMA adiciona automaticamente métricas diferentes para todas as métricas calculadas durante a avaliação. Essas métricas recebem o nome da métrica correspondente, mas com `_diff` anexado ao nome da métrica.

Vamos dar uma olhada nas métricas produzidas por nossa execução:

In [ ]:
tfma.view.render_time_series(eval_result_with_validation)

Agora vamos ver a saída de nossas verificações de validação. Para visualizar os resultados da validação usamos [`tfma.load_validator_result`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/load_validation_result). Para o nosso exemplo, a validação falha porque a AUC está abaixo do threshold.

In [ ]:
validation_result = tfma.load_validation_result(validation_output_path)
print(validation_result.validation_ok)

# Copyright © 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Observação: Este site fornece aplicativos que utilizam dados que foram modificados para uso a partir de sua fonte original obtida em www.cityofchicago.org, site oficial da cidade de Chicago. A cidade de Chicago não faz nenhuma reivindicação quanto ao conteúdo, precisão, atualidade ou integridade de qualquer um dos dados fornecidos neste site. Os dados fornecidos neste site estão sujeitos a alterações a qualquer momento. Entende-se que os dados fornecidos neste site são utilizados por sua conta e risco.